In [118]:
def predict(x):
    from pyspark.sql.functions import substring,conv,col,min,max,udf,lit
   
    d1 = spark.read.csv('/user/maxnet/database/sig.db/term_desc_all',sep='\x01')
    d1 = d1.withColumnRenamed('_c0','term_id')\
           .withColumnRenamed('_c1','brand')\
           .withColumnRenamed('_c2','cn_name')\
           .withColumnRenamed('_c3','en_name')\
           .withColumnRenamed('_c4','type1')\
           .withColumnRenamed('_c5','type2')\
           .withColumnRenamed('_c6','os')\
           .withColumnRenamed('_c7','dtime')\
           .withColumnRenamed('_c8','price')\
           .withColumnRenamed('_c9','remarks')\
           .withColumnRenamed('_c10','prio')

    d2 = spark.read.csv('/user/maxnet/database/sig.db/term_oui',sep='\x01')
    d2 = d2.withColumnRenamed('_c0','mac_6').withColumnRenamed('_c1','manu')
    
    d = spark.read.csv('/user/maxnet/database/sig.db/data_mac_res',sep='\x01')
    d = d.withColumnRenamed('_c0','mac').withColumnRenamed('_c1','id')
    d = d.withColumn('m1_6',substring(d.mac,1,6))

    tmp_1 = d.join(d2,d.m1_6==d2.mac_6)
    df = tmp_1.join(d1,tmp_1.id == d1.term_id).select('mac','id','manu','brand','prio').distinct().dropna()
    df = df.filter(df.prio == 4).select('mac','id','brand').distinct().dropna()
    
    df = df.withColumn('m1',substring(df.mac,1,6)).withColumn('m2',substring(df.mac,7,6))
    df = df.withColumn('p',conv(df.m2, 16, 10))
    df = df.select('mac','id','m1','m2',col('p').cast('bigint'))
    d1 = df.filter(df.m1 =='24DF6A')
    
    gp_id_pmin = d1.groupBy('id').agg({'p':'min'})
    gp_id_pmax = d1.groupBy('id').agg({'p':'max'}).withColumnRenamed('id','id1')
    gp_id_count = d1.groupBy('id').count().withColumnRenamed('id','id2')

    stat = gp_id_pmin.join(gp_id_pmax,gp_id_pmin.id ==gp_id_pmax.id1)
    stat = stat.join(gp_id_count,stat.id == gp_id_count.id2)
    stat = stat.select('id','min(p)','max(p)','count')
    stat = stat.withColumn('effect_density',col('count')/(col('max(p)')-col('min(p)')))
    stat = stat.fillna(0,'effect_density')
    
    
    bottom = d1.select(min('p')).collect()[0]['min(p)']
    top = d1.select(max('p')).collect()[0]['max(p)']
    
    base = stat.withColumn('x-min(p)',lit(x)-col('min(p)'))\
           .withColumn('x-max(p)',lit(x)-col('max(p)'))

    base = base.withColumn('mul',col('x-min(p)')*col('x-max(p)'))

    position = udf(lambda x: 0 if x > 0 else 1)

    base = base.withColumn('position',position('mul'))
    base = base.select('id','min(p)','max(p)','count','effect_density','position').toPandas()   
    
    if x < bottom:
        return base[base['min(p)'] == bottom].iloc[0]['id']
    
    elif x > top:
        return base[base['min(p)'] == top].iloc[0]['id']
    
    else :
        max_effect = base[base.position == '1']['effect_density'].max()
        return base[base.effect_density == max_effect].iloc[0]['id']   

In [119]:
from pyspark.sql.types import StringType
from pyspark.sql.functions import substring,conv,lit,col
test1 = [{'mac':'24DF6A0018CA'}]
test2 = [{'mac':'24DF6A0018CE'}]
inp1 = spark.createDataFrame(test1)
inp2 = spark.createDataFrame(test2)
inp = inp1.union(inp2)
inp = inp.withColumn('m1',substring(inp.mac,1,6)).withColumn('m2',substring(inp.mac,7,6)).withColumn('id',lit(None).cast(StringType()))
inp = inp.withColumn('p',conv(inp.m2, 16, 10))  
inp = inp.select('mac','m1','m2',col('p').cast('bigint'),'id')

/usr/lib/spark/python/pyspark/sql/session.py:357: UserWarning: inferring schema from dict is deprecated,please use pyspark.sql.Row instead
  warnings.warn("inferring schema from dict is deprecated,"


In [121]:
result = []
for x in inp.select('p').toPandas()['p'].values.tolist():
    result.append(predict(x))

In [122]:
result

['40316', '40316']

In [123]:
import pandas as pd
mac = inp.select('mac').toPandas()['mac'].values.tolist()
spark.createDataFrame(pd.DataFrame({'mac':mac,'id':result})).show()

+------------+-----+
|         mac|   id|
+------------+-----+
|24DF6A0018CA|40316|
|24DF6A0018CE|40316|
+------------+-----+



# Test

In [124]:
from pyspark.sql.functions import substring,conv,col,min,max,udf,lit
   
d1 = spark.read.csv('/user/maxnet/database/sig.db/term_desc_all',sep='\x01')
d1 = d1.withColumnRenamed('_c0','term_id')\
       .withColumnRenamed('_c1','brand')\
       .withColumnRenamed('_c2','cn_name')\
       .withColumnRenamed('_c3','en_name')\
       .withColumnRenamed('_c4','type1')\
       .withColumnRenamed('_c5','type2')\
       .withColumnRenamed('_c6','os')\
       .withColumnRenamed('_c7','dtime')\
       .withColumnRenamed('_c8','price')\
       .withColumnRenamed('_c9','remarks')\
       .withColumnRenamed('_c10','prio')

d2 = spark.read.csv('/user/maxnet/database/sig.db/term_oui',sep='\x01')
d2 = d2.withColumnRenamed('_c0','mac_6').withColumnRenamed('_c1','manu')

d = spark.read.csv('/user/maxnet/database/sig.db/data_mac_res',sep='\x01')
d = d.withColumnRenamed('_c0','mac').withColumnRenamed('_c1','id')
d = d.withColumn('m1_6',substring(d.mac,1,6))

tmp_1 = d.join(d2,d.m1_6==d2.mac_6)
df = tmp_1.join(d1,tmp_1.id == d1.term_id).select('mac','id','manu','brand','prio').distinct().dropna()
df = df.filter(df.prio == 4).select('mac','id','brand').distinct().dropna()

df = df.withColumn('m1',substring(df.mac,1,6)).withColumn('m2',substring(df.mac,7,6))
df = df.withColumn('p',conv(df.m2, 16, 10))
df = df.select('mac','id','m1','m2',col('p').cast('bigint'))
d1 = df.filter(df.m1 =='702ED9')

In [125]:
def predict(x):
    from pyspark.sql.functions import substring,conv,col,min,max,udf,lit
   
    d1 = spark.read.csv('/user/maxnet/database/sig.db/term_desc_all',sep='\x01')
    d1 = d1.withColumnRenamed('_c0','term_id')\
           .withColumnRenamed('_c1','brand')\
           .withColumnRenamed('_c2','cn_name')\
           .withColumnRenamed('_c3','en_name')\
           .withColumnRenamed('_c4','type1')\
           .withColumnRenamed('_c5','type2')\
           .withColumnRenamed('_c6','os')\
           .withColumnRenamed('_c7','dtime')\
           .withColumnRenamed('_c8','price')\
           .withColumnRenamed('_c9','remarks')\
           .withColumnRenamed('_c10','prio')

    d2 = spark.read.csv('/user/maxnet/database/sig.db/term_oui',sep='\x01')
    d2 = d2.withColumnRenamed('_c0','mac_6').withColumnRenamed('_c1','manu')
    
    d = spark.read.csv('/user/maxnet/database/sig.db/data_mac_res',sep='\x01')
    d = d.withColumnRenamed('_c0','mac').withColumnRenamed('_c1','id')
    d = d.withColumn('m1_6',substring(d.mac,1,6))

    tmp_1 = d.join(d2,d.m1_6==d2.mac_6)
    df = tmp_1.join(d1,tmp_1.id == d1.term_id).select('mac','id','manu','brand','prio').distinct().dropna()
    df = df.filter(df.prio == 4).select('mac','id','brand').distinct().dropna()
    
    df = df.withColumn('m1',substring(df.mac,1,6)).withColumn('m2',substring(df.mac,7,6))
    df = df.withColumn('p',conv(df.m2, 16, 10))
    df = df.select('mac','id','m1','m2',col('p').cast('bigint'))
    d1 = df.filter(df.m1 =='702ED9')
    
    gp_id_pmin = d1.groupBy('id').agg({'p':'min'})
    gp_id_pmax = d1.groupBy('id').agg({'p':'max'}).withColumnRenamed('id','id1')
    gp_id_count = d1.groupBy('id').count().withColumnRenamed('id','id2')

    stat = gp_id_pmin.join(gp_id_pmax,gp_id_pmin.id ==gp_id_pmax.id1)
    stat = stat.join(gp_id_count,stat.id == gp_id_count.id2)
    stat = stat.select('id','min(p)','max(p)','count')
    stat = stat.withColumn('effect_density',col('count')/(col('max(p)')-col('min(p)')))
    stat = stat.fillna(0,'effect_density')
    
    
    bottom = d1.select(min('p')).collect()[0]['min(p)']
    top = d1.select(max('p')).collect()[0]['max(p)']
    
    base = stat.withColumn('x-min(p)',lit(x)-col('min(p)'))\
           .withColumn('x-max(p)',lit(x)-col('max(p)'))

    base = base.withColumn('mul',col('x-min(p)')*col('x-max(p)'))

    position = udf(lambda x: 0 if x > 0 else 1)

    base = base.withColumn('position',position('mul'))
    base = base.select('id','min(p)','max(p)','count','effect_density','position').toPandas()   
    
    if x < bottom:
        return base[base['min(p)'] == bottom].iloc[0]['id']
    
    elif x > top:
        return base[base['min(p)'] == top].iloc[0]['id']
    
    else :
        max_effect = base[base.position == '1']['effect_density'].max()
        return base[base.effect_density == max_effect].iloc[0]['id']   

In [135]:
d1.sample(0.001).show()

+------------+-----+------+------+-------+
|         mac|   id|    m1|    m2|      p|
+------------+-----+------+------+-------+
|702ED907022B|42980|702ED9|07022B| 459307|
|702ED91411FE|42980|702ED9|1411FE|1315326|
|702ED93DA3CF|39000|702ED9|3DA3CF|4039631|
|702ED94BC16E|42980|702ED9|4BC16E|4964718|
+------------+-----+------+------+-------+



In [137]:
d1.sample(0.001).select('p').toPandas()['p'].values.tolist()

[3536723, 967250, 3553025, 2836777, 15370372, 458578]

In [153]:
test = d1.sample(0.01,seed=1)

In [154]:
test.show()

+------------+-----+------+------+--------+
|         mac|   id|    m1|    m2|       p|
+------------+-----+------+------+--------+
|702ED941EC07|48993|702ED9|41EC07| 4320263|
|702ED93F3427|48993|702ED9|3F3427| 4142119|
|702ED9144F0B|42980|702ED9|144F0B| 1330955|
|702ED9915C23|40232|702ED9|915C23| 9526307|
|702ED92B67FF|48993|702ED9|2B67FF| 2844671|
|702ED9EA580E|10666|702ED9|EA580E|15357966|
|702ED9EA0451|10666|702ED9|EA0451|15336529|
|702ED93FDBC2|42980|702ED9|3FDBC2| 4185026|
|702ED940DAE0|48993|702ED9|40DAE0| 4250336|
|702ED923E9D8|42980|702ED9|23E9D8| 2353624|
|702ED9412DEE|48993|702ED9|412DEE| 4271598|
|702ED908D72B|42980|702ED9|08D72B|  579371|
|702ED90DCB9B|42980|702ED9|0DCB9B|  904091|
|702ED9BC4C7D|40320|702ED9|BC4C7D|12340349|
|702ED911D01A|42980|702ED9|11D01A| 1167386|
|702ED906FC7A|42980|702ED9|06FC7A|  457850|
|702ED93F361D|48993|702ED9|3F361D| 4142621|
|702ED923C0F3|42980|702ED9|23C0F3| 2343155|
|702ED976A02E|10666|702ED9|76A02E| 7774254|
|702ED907FFC1|42980|702ED9|07FFC

In [ ]:
import pandas as pd
result = []
for x in test.select('p').toPandas()['p'].values.tolist():
    result.append(predict(x))

mac = test.select('mac').toPandas()['mac'].values.tolist()
spark.createDataFrame(pd.DataFrame({'mac':mac,'id':result})).show()

In [ ]:
test.filter(test.mac == '702ED913E0C2').show()